## Exploring housing markers in Canada and British Columbia


Average value of new mortgage loan 

Canada, provinces and CMAs 2012 Q3 to 2020 Q3

In [ ]:
%run ./scr/main.py
import plotly.express as px

In [ ]:
data = "./data/"

In [ ]:
# a function for data cleaning
def data_prep(data_file):
    #Reading the data and making a copy
    df = pd.read_excel(data_file)
    temp = df.copy()

    #Removing empty rows
    temp.dropna(thresh=2, inplace=True)

    #Reseting columns - Changing the column names using the values in the 1st row
    temp_header = temp.iloc[0,]
    temp = temp[1:]
    temp.columns = temp_header

    #Removing all columns whose names are nan
    temp = temp.loc[:, temp.columns.notnull()]

    #Ensuring the indices go from 0 without skipping any number
    temp.reset_index(inplace=True, drop = True)

    #Categorize Geography as Country, Province, and City
    temp['Region/City'] = ""
    temp['Region/City'][temp.index ==0] = 'Country'
    temp['Region/City'][(temp.index < 11) & (temp.index >0)] = 'Province'
    temp['Region/City'][temp.index >= 11] = 'City'

    #Rearrange columns -- bring column 'Region/City' to the 1st column position
    # Access columns
    cols = temp.columns.tolist()
    # Bring last col to front
    cols = cols[-1:] + cols[:-1]
    # Set new column order
    temp = temp[cols] 

    #Converting data type to the right format
    #Isolating yearly and quarterly columns and convert to numeric.
    int_cols = temp.columns.drop(['Region/City', 'Geography'])
    temp[int_cols] = temp[int_cols].apply(pd.to_numeric, errors='coerce')

    return temp

In [ ]:
# a function for preparing the data to the right shape
def slice_data(df, level):
    """
    df: data frame with mortgage data
    level: "Province" or "City"
    Extract a subset of df based on level
    Return a dataframe
    """
    try:
        temp = df[df['Region/City']==level]
        temp = pd.melt(temp, id_vars='Geography', value_vars=temp.columns[2:])
        temp.rename(columns = {3:'Time'}, inplace = True)
        return temp
    except KeyError:
        print("Key not found. Make sure that 'level' is in ['Province','City']")


In [ ]:
# a function for box or violin graphs
def graph_region(region_df, graph_type: str, title):
    """
    region_df: reshaped data frame object with mortage data
    graph_type: "box", "violin"
    title: title of the graph
    """
    
    plot_dict = {'box': px.box,'violin': px.violin}
        
    try:
        fig = plot_dict[graph_type](region_df, x="Geography", y="value", color = "Geography", points='all')
        fig.update_layout(title = title)
        fig.update_xaxes(tickangle=-45)
        fig.show()
    
    except KeyError:
        print("Key not found. Make sure that 'graph_type' is in ['box','violin']")

# Mortgage loan

In [ ]:
data_file = data + "average-value-new-mortgage-loans-ca-prov-cmas-2012-q3-2020-q3-en.xlsx"
df_mortgage = data_prep(data_file)
level = 'Province'
df_mortgage_long = slice_data(df_mortgage, level = level)

In [ ]:
graph_type = 'box'
graph_region(df_mortgage_long, graph_type=graph_type, title = f'{graph_type} plot of mortgage loans by {level}'.upper())

In [ ]:
#line plot
fig = px.line(df_mortgage_long, x='Time', y='value', color = 'Geography',
              title = f'Line plot of mortgage loans by {level}'.upper())
fig.update_xaxes(tickangle=-45)
fig.show()

# Delinquency

In [ ]:
data_delinquency = data + "mortgage-delinquency-rate-ca-prov-cmas-2012-q3-2020-q4-en.xlsx"
df_delin = data_prep(data_delinquency)

In [ ]:
level='Province'
graph_type='box'
title = f'{graph_type} plot of delinquency rate by {level}'.upper()
df_delin_long = slice_data(df_delin, level = level)
graph_region(df_delin_long, graph_type=graph_type, title = title)

# Average Income

In [ ]:
data_income = data + "real-average-household-income-after-taxes-tenure-2006-2018-en.xlsx"
df_income = data_prep(data_income)

In [ ]:
level='Province'
graph_type='box'
title = f'{graph_type} plot of average household income by {level}'.upper()
df_income_long = slice_data(df_income, level = level)
graph_region(df_income_long, graph_type=graph_type, title = title)

In [ ]:
#line plot
fig = px.line(df_income_long, x='Time', y='value', color = 'Geography',
              title = f'Line plot of avergage household income by {level}'.upper())
#fig.update_xaxes(tickangle=-45)
fig.show()

# Combining 2 data sets - mortgage loans and household income

In [ ]:
#set(df['Geography']).difference(df_income['Geography'])
#len(set(df['Geography']).difference(df_income['Geography']))
#set(df[df['Region/City']=='Province']['Geography']).difference(df_income[df_income['Region/City']=='Province']['Geography'])

In [ ]:
mortgage = df_mortgage.copy()

# change the name of 2 provinces to make the provinces match in 2 datasets
mortgage_replace = {'Newfoundland':'Newfoundland and Labrador', 'Québec': 'Quebec'}
mortgage.replace(mortgage_replace, inplace=True)

# create yearly data for mortgage for some years
select_year = df_income.columns.tolist()
select_year = select_year[select_year.index(2013.0):]
select_year = [str(int(y)) for y in select_year]

# average over 4 quarters to get the yearly value
q = ['Q1', 'Q2', 'Q3', 'Q4']
for y in select_year:
    y1 = [y+j for j in q]
    mortgage[y] = mortgage[y1].mean(axis=1)
    
# select columns in mortgage data
col_mortgage = ['Region/City', 'Geography']+select_year
mortgage13_18 = mortgage[col_mortgage]

In [ ]:
col_income = ['Region/City', 'Geography']+[float(y) for y in select_year]
mortgage13_18.columns = col_income

income = df_income.copy()
income13_18 = income[col_income]

In [ ]:
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [ ]:
mortgage13_18['Type'] = 'Mortgage'
income13_18['Type'] = 'Income'

In [ ]:
df13_18 = pd.concat([mortgage13_18, income13_18])

In [ ]:
df13_18.reset_index(inplace=True, drop=True)
df_province = df13_18[df13_18['Region/City']=='Province']

# Mortgage loan and household income

In [ ]:
df_province_melt = pd.melt(df_province, id_vars=['Geography','Type'], value_vars=df_province.columns[2:-1])
#df_province.pivot(index = 'Geography', columns = 'Type', values=[2013.0 + i for i in range(6)])

In [ ]:
df_province.head()

In [ ]:
pivot_province_income = df_province.pivot_table(columns=["Geography", "Type"])

In [ ]:
pivot_province_income

In [ ]:
import matplotlib.pyplot as plt

provinces = set([item[0] for item in pivot_province_income.columns])

for base_loc in provinces:
    
    base_location = pivot_province_income[base_loc]
    income = base_location['Income']
    mortgage = base_location['Mortgage']

    plt.scatter(income, mortgage, label=base_loc)

plt.xlabel("Income")
plt.ylabel("Mortgage")
plt.title("Mortgage vs Income per Province")
plt.legend(loc="best", bbox_to_anchor=(1.55, 1.03))
plt.grid(True)
plt.show()

Alberta has the highest average income yet their mortgage remains stable. 

Ontario and British Columbia show the highest mortgage prices. 



In [ ]:
pivot_province_income

In [ ]:
pivot_province_income.T.index==('Alberta', 'Income')

In [ ]:
pivot_province_income

### PIVOT ORDER MATTERS

In [ ]:
pivot_income_province = df_province.pivot_table(columns=["Type", "Geography"])

pivot_income_province

In [ ]:
pivot_income_province.Income

In [ ]:
# mor 100
# inc what percentage of my mortgage is my income

fig = px.scatter(pivot_income_province.Income/ pivot_income_province.Mortgage*100, 
           facet_col="Geography",
          title="Income to mortgage by province")

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

In [ ]:
# inco 
# mor what percentage of income is my mortgage 

fig = px.scatter(pivot_income_province.Mortgage/ pivot_income_province.Income , 
                 facet_col="Geography", title='Mortgage to income by province')
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

In [ ]:
mort_prov = mortgage13_18[mortgage13_18['Region/City']=='Province'].pivot_table(columns='Geography')

In [ ]:
inco_prov = income13_18[income13_18['Region/City']=='Province'].pivot_table(columns='Geography')

In [ ]:
import seaborn as sns


In [ ]:
sns.pairplot(df_province,hue='Type')


Strong relationship between any given pair of years.

In [ ]:

sns.pairplot(df_province,hue='Geography')


Strong relationship between any given pair of years, bimodal distribution for each province.

In [ ]:
sns.catplot(y=2013.0, x="Geography", hue="Type", kind="bar", data=df_province)
plt.xticks(rotation=90)
sns.catplot(y=2013.0, x="Type", hue="Geography", kind="bar", data=df_province)
plt.xticks(rotation=45)
plt.show()

## Exercise

1. Identify different ways of visualizing data. 
2. What insights do you gain for the study period? 
3. How are these insights different for each province/city?